<a href="https://colab.research.google.com/github/jasreman8/OOPs-for-Intelligent-Agentic-Systems/blob/main/Polymorphism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

- Understand the concept of polymorphism in Object-Oriented Programming.
- Be introduced to the idea of an "interface" (or "protocol") as a contract for behavior.
- See how LangChain's Runnable serves as a core interface.
- Appreciate how polymorphism enables different components (LLMs, Prompts, Parsers, Retrievers) to be used interchangeably within LangChain Expression Language (LCEL) chains, as long as they adhere to the Runnable interface.

# Introduction to Interfaces

We've learned about inheritance, where a child class gets features from a parent. Polymorphism (from Greek meaning "many forms") is a principle that often goes hand-in-hand with inheritance, but it's really about objects of different classes being treatable through a common interface.

Imagine you have different types of payment methods: Credit Card, PayPal, Bank Transfer. You, as the merchant, just want to call a process_payment(amount) method. You don't want to write separate code for each payment type if the core action is the same. Polymorphism allows this.

In Python, an "interface" isn't a strict keyword like in some other languages (e.g., Java's interface). Instead, it's more of a contract or a protocol. It defines a set of methods that a class promises to implement.

If a class implements all the methods defined by an interface, it's said to "adhere to" or "satisfy" that interface.
This means you can expect objects of that class to respond to those specific method calls.

**Duck Typing (Python's Approach):**

Python often uses "duck typing": "If it walks like a duck and quacks like a duck, then it must be a duck."
If an object has the methods you need (e.g., an `.invoke()` method), you can often use it as if it adheres to an expected interface, regardless of its specific class or inheritance hierarchy.

**LangChain's `Runnable` as a Core Interface:**

In LangChain, the `langchain_core.runnables.Runnable` class (and its associated protocols) acts as a fundamental interface. Most components designed to be part of an LCEL chain (Prompts, LLMs, Output Parsers, Retrievers, custom functions wrapped with RunnableLambda, etc.) adhere to the Runnable interface.

This "contract" means they are expected to have certain methods, most notably:
- `.invoke(input, config=None)`: Execute the Runnable with a single input.
- `.stream(input, config=None)`: Stream the output for a single input.
- `.batch(inputs, config=None)`: Execute with a batch of inputs.

You don't usually create an object of `Runnable` itself. Instead, classes like `ChatPromptTemplate`, `ChatOpenAI`, `StrOutputParser` implement the `Runnable` interface (often by inheriting from base classes that themselves implement `Runnable`).


# Polymorphism in Action: The `Runnable` Interface

As we have seen, any component that is a `Runnable` promises to implement a set of common methods. Let us look more closely at the most important one for basic execution - `.invoke(input, config=None)`: This method is the primary way to execute a `Runnable` with a single input and get a single output.

Let's look at how different LangChain components, all being Runnables, implement .invoke() in their own specialized way.

In [1]:
! pip install -q langchain-openai==0.3.24

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.3 MB/s eta 0:00:00


In [2]:
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from google.colab import userdata

In [3]:
# --- Initialize our components ---
# Each of these is a 'Runnable' and thus has an .invoke() method

# 1. ChatPromptTemplate: A Runnable
prompt_runnable = ChatPromptTemplate.from_template(
    "Generate a professional subject line for an email about {topic}."
)

In [4]:
# 2. ChatOpenAI (LLM): A Runnable
llm_runnable = ChatOpenAI(
    api_key=userdata.get('OPEN_API_KEY'),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model="gpt-4o-mini",
    temperature=0.5
)

In [5]:
# 3. StrOutputParser: A Runnable
parser_runnable = StrOutputParser()

In [6]:
# --- Demonstrating .invoke() on each Runnable component ---

# A. Invoking the ChatPromptTemplate

prompt_input = {"topic": "a new product launch for eco-friendly packaging"}
formatted_prompt = prompt_runnable.invoke(prompt_input)

print(f"Input to prompt: {prompt_input}")
print(f"\nOutput of prompt.invoke() (type: {type(formatted_prompt)}):")
# For ChatPromptTemplate, the result is usually a ChatPromptValue, whose .to_messages() gives list of BaseMessage
for message in formatted_prompt.to_messages():
    print(f"  - ({message.type}) {message.content}")

Input to prompt: {'topic': 'a new product launch for eco-friendly packaging'}

Output of prompt.invoke() (type: <class 'langchain_core.prompt_values.ChatPromptValue'>):
  - (human) Generate a professional subject line for an email about a new product launch for eco-friendly packaging.


In [11]:
# B. Invoking the ChatOpenAI LLM
formatted_prompt = prompt_runnable.invoke({"topic": "a new product launch for eco-friendly packaging"})
print("\n--- 2. Invoking ChatOpenAI (LLM) ---\n")
# What does .invoke() do for an LLM? It sends the prompt to the AI and gets a response.
# The input to an LLM is typically the output of a prompt (a list of messages or a string).
llm_response_message = llm_runnable.invoke(formatted_prompt.to_messages())
print(f"Input to LLM (type: {type(formatted_prompt.to_messages())}): A list of messages")
print(f"\nOutput of llm.invoke() (type: {type(llm_response_message)}):")
print(f"  - ({llm_response_message.type}) Content: '{llm_response_message.content}'")


--- 2. Invoking ChatOpenAI (LLM) ---

Input to LLM (type: <class 'list'>): A list of messages

Output of llm.invoke() (type: <class 'langchain_core.messages.ai.AIMessage'>):
  - (ai) Content: '"Introducing Our Innovative Eco-Friendly Packaging Solutions: Join the Green Revolution!"'


In [12]:
# C. Invoking the StrOutputParser
print("\n--- 3. Invoking StrOutputParser ---")
# What does .invoke() do for StrOutputParser? It takes an AI Message (or similar) and extracts the string content.
# The input to the parser is typically the output of an LLM.
parsed_string_output: str = parser_runnable.invoke(llm_response_message)
print(f"\nInput to parser (type: {type(llm_response_message)}): An AIMessage object")
print(f"\nOutput of parser.invoke() (type: {type(parsed_string_output)}): '{parsed_string_output}'")


--- 3. Invoking StrOutputParser ---

Input to parser (type: <class 'langchain_core.messages.ai.AIMessage'>): An AIMessage object

Output of parser.invoke() (type: <class 'str'>): '"Introducing Our Innovative Eco-Friendly Packaging Solutions: Join the Green Revolution!"'


Now, the magic of LangChain Expression Language (LCEL)! The | (pipe) operator in LCEL is designed to work with Runnables. It leverages polymorphism.
When you write:

```python
chain: Runnable = prompt_runnable | llm_runnable | parser_runnable
```

1. ```prompt_runnable | llm_runnable```:
    - LCEL knows both are Runnables.
    - It creates a sequence. When this sequence is invoked (e.g., with `chain.invoke(prompt_input)`):
        - It first calls `prompt_runnable.invoke(prompt_input)`. The output is `formatted_prompt_value`.
        - Then, it takes `formatted_prompt_value` and calls `llm_runnable.invoke(formatted_prompt_value.to_messages())`. The output is `llm_response_message`.
        - The intermediate `RunnableSequence` (from `prompt_runnable | llm_runnable`) now has `llm_response_message` as its result.

2. `(prompt_runnable | llm_runnable) | parser_runnable`:
    - The `RunnableSequence` from step 1 is itself a Runnable.
    - Its output (`llm_response_message`) is then piped to `parser_runnable`.
    - `parser_runnable.invoke(llm_response_message)` is called, resulting in `parsed_string_output`.

The | operator doesn't need to know the specific details of how each component implements invoke(). It only needs to trust that:

- Each component has an `invoke()` method (satisfies the `Runnable` interface).
- The output type of one component's `invoke()` is compatible with the input type of the next component's `invoke()`. (LangChain often handles common coercions, but type compatibility is important).

In [13]:
# --- Composing and Invoking the Full LCEL Chain ---
email_subject_chain = prompt_runnable | llm_runnable | parser_runnable

print("\n--- Invoking the Full LCEL Chain ---")
final_email_subject: str = email_subject_chain.invoke(prompt_input) # Same input as for the prompt alone

print(f"\nInput to chain: {prompt_input}")
print(f"\nOutput of chain.invoke() (type: {type(final_email_subject)}): '{final_email_subject}'")


--- Invoking the Full LCEL Chain ---

Input to chain: {'topic': 'a new product launch for eco-friendly packaging'}

Output of chain.invoke() (type: <class 'str'>): '"Introducing Our Innovative Eco-Friendly Packaging Solutions: Join Us in Making a Sustainable Impact!"'
